REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [63]:
import pandas as pd

In [64]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [65]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [66]:
ins.head(5)

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [67]:
print("records:"+str(len(bus)))


records:6315


#### Question 2b: How many unique business IDs are there?  

In [68]:
print("unique business IDs:"+str(len(bus['business_id'].unique())))

unique business IDs:6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [69]:
print(bus['name'].value_counts()[:5])

STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [70]:
# examine a particular entry
postal_code_1=bus.loc[1,'postal_code']
print(postal_code_1)

94109


In [71]:
print(type(postal_code_1))

<class 'str'>


#### Question 3b

What are the unique values of postal_code?

In [72]:
print(bus['postal_code'].unique())

['94104' '94109' '94133' '94110' '94122' '94103' '94115' '94131' '94111'
 '94117' '94107' '94108' '94102' '94132' '94105' '94134' '94124' '94116'
 '94121' '94118' '94112' '94123' '94114' '94127' '941102019' '941' '94545'
 '94066' '941033148' nan '94158' '95105' '94013' '94130' 'CA' '92672'
 '94120' '94143' '94609' '94101' '00000' '94188' '94621' '94014' '94129'
 '94602' 'Ca']


#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [73]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [74]:
vb = bus[bus['postal_code'].isin(validZip)]
print("numbers of businesses:"+str(len(vb)))

numbers of businesses:5999


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [75]:
vb[pd.isnull(vb['longitude'])]
print("numbers of businesses are missing longitude values:"+str(len(vb[pd.isnull(vb['longitude'])])))

numbers of businesses are missing longitude values:2483


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [76]:
# count the values of null_df and not_null_df
null_df=vb[pd.isnull(vb['longitude'])][['postal_code','business_id']].groupby('postal_code').count()
not_null_df=vb[['postal_code','longitude']].groupby('postal_code').count()
# merge the dataframe
new_df=pd.merge(null_df, not_null_df, left_index=True, right_index=True)
# rename the columns
new_df.rename(columns={'longitude':'not_null_lon','business_id':'null_lon'},inplace=True)
new_df

,null_lon,not_null_lon
postal_code,,
94102,196,262
94103,252,306
94104,59,74
94105,100,126
94107,247,204
94108,82,142
94109,146,237
94110,251,334
94111,115,167


4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [77]:
# the proportion of missing longitude
new_df['proportion']=new_df['null_lon']/(new_df['null_lon']+new_df['not_null_lon'])
new_df

,null_lon,not_null_lon,proportion
postal_code,,,
94102,196,262,0.427948
94103,252,306,0.451613
94104,59,74,0.443609
94105,100,126,0.442478
94107,247,204,0.547672
94108,82,142,0.366071
94109,146,237,0.381201
94110,251,334,0.429060
94111,115,167,0.407801


In [78]:
# the top five zipcodes in descending order of proportion missing postal_code
new_df.sort_values('proportion', ascending = False).head(5)

,null_lon,not_null_lon,proportion
postal_code,,,
94107,247,204,0.547672
94124,88,76,0.536585
94132,65,69,0.485075
94114,108,122,0.469565
94122,123,147,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [79]:
print('rows in the dataset:'+str(len(ins)))
print('unique business:' + str(len(ins['business_id'].unique())))

rows in the dataset:15430
unique business:5730


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [80]:
# values of type
print(ins['type'].unique())
# numbers of each value in the DataFrame
print(ins['type'].value_counts())

['routine' 'complaint']
routine      15429
complaint        1
Name: type, dtype: int64


In [81]:
# the new dataframe: remove the value of complaint to eliminate the rare types（ not sure about the method ）
ins2 = ins[ins.type == 'routine']
ins2

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine
...,...,...,...,...
15425,89515,98,20161103,routine
15426,89701,100,20161206,routine
15427,89790,96,20161129,routine
15428,89803,100,20161215,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [82]:
ins['date']=ins['date'].astype(str)
ins['year']=ins['date'].str.slice(0,4)
ins

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [83]:
# range of years
print("range of years:"+ str(ins['year'].unique()))
# same number of inspections each year or not
inspections_numbers=ins.groupby('year').count()
print(inspections_numbers['business_id'])

range of years:['2016' '2014' '2015' '2013']
year
2013      38
2014    5629
2015    3923
2016    5840
Name: business_id, dtype: int64


In [84]:
# create the a new list without 2013
new_list=inspections_numbers[inspections_numbers['business_id']>50].index.tolist()
# create the new dataframe: ins3
ins3 = ins[ins.year.isin(new_list)]
ins3

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [85]:
# inspections for 2016
ins2016=ins3[ins3.year == '2016']
ins2016

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
3,19,94,20160513,routine,2016
6,24,98,20161005,routine,2016
7,24,96,20160311,routine,2016
10,31,84,20160816,routine,2016
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [86]:
merge_df = pd.merge(bus, ins2016, on='business_id')
merge_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84,20160816,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [87]:
# find 20 lowest rated businesses
merge_df1= merge_df.sort_values('score').head(20)
# print their names, their addresses, and their ratings
merge_df2= merge_df1[['name','address','score']]
merge_df2

,name,address,score
3931,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52
5727,POKI TIME,2101 LOMBARD ST,55
4981,CRAZY PEPPER,2257 SAN JOSE AVE,55
3917,UNCLE CAFE,65 WAVERLY PL,55
2488,HING WANG BAKERY,339 JUDAH ST,56
4481,GOLDEN WOK,295 B ORIZABA AVE,56
4298,TOP'S CAFE,2460 SAN BRUNO AVE,57
4824,L & G VIETNAMESE SANDWICH,602 EDDY ST,57
696,RED A BAKERY,634 CLEMENT ST,57
3415,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!